### Задание

1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки `Sklearn`. В данных оставьте только 2 класса: `Iris Versicolor`, `Iris Virginica`.
2. Самостоятельно реализуйте логистическую регрессию, без использования метода `LogisticRegression` из библиотеки. Можете использовать библиотеки `pandas`, `numpy`, `math` для реализации. Оформите в виде функции. *Оформите в виде класса с методами.*
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
4. Повторите п. 3 для метода скользящего среднего (`Root Mean Square Propagation, RMSProp`).
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (`Nesterov–accelerated Adaptive Moment Estimation, Nadam`).
6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [2]:
# Загрузим датасет Iris
iris = load_iris()
X = iris.data
y = iris.target

# Оставим только два класса: Iris Versicolor (1) и Iris Virginica (2)
X = X[y != 0]
y = y[y != 0]
y = np.where(y == 1, 0, 1)

In [3]:
# Реализуем логистическую регрессию в виде класса с различными методами оптимизации
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weights = None
        self.bias = None
        self.s = None
        self.m = None
        self.v = None        

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    # Обучение модели
    def fit(self, X, y, optimizer='gradient_descent'):
        num_samples, num_features = X.shape
        
        self.bias = 0
        self.weights = np.zeros(num_features)
        self.s = np.zeros(num_features)
        self.m = np.zeros(num_features)
        self.v = np.zeros(num_features)
        
        for t in range(1, self.n_iterations+1):
            
            # Вычисление линейной модели и прогноза модели с помощью сигмоиды
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)
            
            # Вычисление градиентов весов и смещения
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            # Обновление весов и смещения с помощью выбранного оптимизатора
            if optimizer == 'gradient_descent':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
                
            elif optimizer == 'rmsprop':
                self.s = self.beta1 * self.s + (1 - self.beta1) * np.square(dw)
                self.weights -= self.learning_rate * dw / (np.sqrt(self.s) + self.epsilon)
                self.bias -= self.learning_rate * db
                
            elif optimizer == 'nadam':
                self.m = self.beta1 * self.m + (1 - self.beta1) * dw
                self.v = self.beta2 * self.v + (1 - self.beta2) * np.square(dw)
            
                m_hat = self.m / (1 - self.beta1 ** t)
                v_hat = self.v / (1 - self.beta2 ** t)
            
                self.weights -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
                self.bias -= self.learning_rate * db
    
    # Прогноз метки классов для новых данных на основе обученной модели
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        return np.round(y_pred)
    
    # Вычисление точность модели
    def accuracy(self, y_true, y_pred):
        return np.mean(y_true == y_pred)

In [4]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Выполним стандартизацию признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Инициализируем и обученим модель с различными методами оптимизации
methods = {'Gradient Descent':'gradient_descent', 'RMSProp':'rmsprop', 'Nadam':'nadam'}
results = {}

for name, method in methods.items():
    start_time = time.time()
    model = MyLogisticRegression()
    model.fit(X_train, y_train, optimizer=method)
    end_time = time.time()
    
    y_pred = model.predict(X_test)
    accuracy = model.accuracy(y_test, y_pred)
    runtime = end_time - start_time
    
    results[name] = {'Accuracy': accuracy, 'Runtime': runtime}

In [7]:
res_df = pd.DataFrame.from_dict(results, orient='index')
print(res_df)

                  Accuracy   Runtime
Gradient Descent      0.85  0.033039
RMSProp               0.85  0.039028
Nadam                 0.85  0.048044


### Вывод:

В ходе работы была осуществлена реализация логистической регрессии и трёх методов оптимизации (`градиентный спуск`, `RMSProp`, `Nadam`). Для каждого метода выполнен подсчёт выбранной метрики качества (`accuracy`), а также произведён замер времени работы. Результаты представлены в виде таблицы.

Можно заметить, что все методы достигли одинакового значения метрики качества и лишь незначительно отличаются по скорости работы. 